<a href="https://colab.research.google.com/github/arcyda/DeepLearningHandsOn/blob/main/linear_reg_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import torch



In [ ]:
A = torch.ones((3,2))
print(A)
B = A.reshape((-1,1))
print(B), B.shape

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


(None, torch.Size([6, 1]))

第一步: 生成数据集

In [15]:
def synthetic_data(w, b, num_examples):
  X = torch.normal(0,1, (num_examples, len(w))) #normal() 生成平均值为0，方差为1的（m,n）m行n列矩阵
  y = torch.matmul(X, w) + b #matmul() 两个矩阵相乘 X:1000X2, w:torch.Size([2])
  y += torch.normal(0, 0.01, y.shape)
  return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000) #features每一行为一个样本，labels每一行为target的特征值


In [16]:
features.shape, labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [17]:
print('features:', features[0], '\nlabel:', labels[0])

features: tensor([-1.2294,  0.2575]) 
label: tensor([0.8736])


第二步：读取数据集

In [18]:
#抽取小批量样本
def data_ite(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples)) #生成采样数组

  random.shuffle(indices) #打乱数组

  for i in range(0, num_examples, batch_size):
    batch_indices = torch.tensor(indices[i : min(i + batch_size, num_examples)]) #选取num_examples行数据 
    yield features[batch_indices], labels[batch_indices]


In [20]:
batch_size = 10

i = 0
for X, y in data_ite(batch_size, features, labels): #遍历所有的小样本
  print('X:\n', X, '\ny:\n', y)
  i = i + 1
  if(i == 2): break

X:
 tensor([[-0.0333,  0.9324],
        [-0.8661, -0.1213],
        [ 1.6917,  0.2248],
        [-0.4201, -0.9824],
        [-0.0216,  1.0184],
        [-0.3059, -0.0933],
        [-0.1948,  0.7328],
        [-0.7983, -1.4287],
        [-1.2101,  0.2056],
        [-1.3501, -1.9026]]) 
y:
 tensor([[0.9516],
        [2.8641],
        [6.8380],
        [6.7088],
        [0.7022],
        [3.8998],
        [1.3418],
        [7.4526],
        [1.0823],
        [7.9449]])
X:
 tensor([[ 4.9553e-01, -6.9358e-01],
        [ 2.1544e-03, -3.2921e-01],
        [-1.9870e+00,  2.8895e-01],
        [-1.3438e-01, -8.4445e-01],
        [ 4.6911e-01, -6.7341e-01],
        [-1.2465e+00, -1.2745e+00],
        [-1.7047e+00, -1.0669e+00],
        [ 1.5188e+00,  2.1600e+00],
        [ 5.8397e-01, -1.9166e-01],
        [ 5.9154e-01,  1.0904e+00]]) 
y:
 tensor([[ 7.5510],
        [ 5.3159],
        [-0.7672],
        [ 6.8114],
        [ 7.4231],
        [ 6.0349],
        [ 4.4410],
        [-0.1103],
       

In [ ]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
grid = torch.tensor([[1,1], [2,2], [3,3]])
l = list(range(5))
b = torch.tensor(l[0:3])
c = l[1:2]
print(l)
print(b)
b.shape
#print(grid[b])

[0, 1, 2, 3, 4]
tensor([0, 1, 2])


torch.Size([3])

第三步：初始化模型参数

In [ ]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

我们的任务：更新参数w, b，直到这些参数足够拟合数据集

注意：每次参数更新，都要计算损失函数关于模型参数的梯度。根据这个梯度值，可以让参数向减小损失的方向更新。如何计算梯度？使用自动微分

第四步：定义模型

In [ ]:
def linreg(X, w, b): #线性回归模型
  return torch.matmul(X, w) + b

第五步：定义损失函数

In [ ]:
def squared_loss(y_hat, y): #差的平方
  return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2 #为什么这里要对y进行reshape??

第六步：定义优化算法